# **Laboratorio 12: 🚀 Despliegue 🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Sebastian Tinoco
- Auxiliar: Catherine Benavides, Consuelo Rojas
- Ayudante: Eduardo Moya, Nicolás Ojeda

### Equipo: 

- Pía Antiquera 
- Evelyn Silva

### **Link de repositorio de GitHub:** `https://github.com/piaantiquera/Courses-MDS7202.git`

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor


X_train, X_test, y_train, y_test = train_test_split(df_x, df_y)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `mean absolute error` se registre como `"valid_mae"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor


X_train, X_test, y_train, y_test = train_test_split(df_x, df_y)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
    

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_mae")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

In [32]:
### 1
import mlflow
import mlflow.xgboost
import xgboost as xgb
import optuna
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

import pandas as pd

path = 'water_potability.csv'
df = pd.read_csv(path, sep=',')
promedios = df.mean()

# Reemplazar los valores nulos por el promedio de cada columna
df.fillna(promedios, inplace=True)

df_x = df.drop(columns=['Potability'])

df_y = df['Potability']

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,test_size=0.2, random_state=42)

# Configurar el directorio para los artefactos
artifacts_dir = "mlruns/artifacts"
os.makedirs(artifacts_dir, exist_ok=True)

# Definir la función de optimización
def objective(trial):
    # Sugerir valores para los hiperparámetros
    params = {
        "booster": "gbtree",
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "mae",
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0)
    }

    # Crear y entrenar el modelo
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)

    # Predecir y calcular el error absoluto medio
    y_pred = model.predict(X_test)
    valid_mae = mean_absolute_error(y_test, y_pred)
    
    # Registrar los resultados en MLflow
    with mlflow.start_run():
        mlflow.log_params(params)
        mlflow.log_metric("valid_mae", valid_mae)
    
    return valid_mae

# Optimizar el modelo con Optuna
def optimize_model():
    # Crear un estudio de Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)

    # Obtener el mejor modelo
    best_trial = study.best_trial
    best_model = xgb.XGBRegressor(**best_trial.params)
    best_model.fit(X_train, y_train)

    # Guardar el mejor modelo
    model_path = "models/best_model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(best_model, f)
    
    # Guardar los gráficos de Optuna en MLflow
    optuna.visualization.plot_optimization_history(study).write_image(f"{artifacts_dir}/optimization_history.png")
    optuna.visualization.plot_param_importances(study).write_image(f"{artifacts_dir}/param_importances.png")

    # Guardar la importancia de las características
    plt.figure(figsize=(10, 6))
    xgb.plot_importance(best_model, importance_type="weight")
    plt.savefig(f"{artifacts_dir}/feature_importance.png")
    plt.close()

    # Registrar las versiones de las librerías
    mlflow.log_artifacts(artifacts_dir)
    mlflow.set_tag("mlflow.version", mlflow.__version__)
    mlflow.set_tag("optuna.version", optuna.__version__)
    mlflow.set_tag("xgboost.version", xgb.__version__)

    return best_model

if __name__ == "__main__":
    best_model = optimize_model()


[I 2024-07-05 04:22:17,833] A new study created in memory with name: no-name-a112296d-aaed-4621-a3c9-3f54e65f53a4
/var/folders/kw/_g4nm_m11vd8530g9t7gvv180000gn/T/ipykernel_44247/2173255088.py:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 1e-1),
/var/folders/kw/_g4nm_m11vd8530g9t7gvv180000gn/T/ipykernel_44247/2173255088.py:42: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
/var/folders/kw/_g4nm_m11vd8530g9t7gvv180000gn/T/ipykernel_44247/2173255088.py:43: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See http

<Figure size 1000x600 with 0 Axes>

In [34]:
!pip install "fastapi[all]"

  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 800.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 1.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 896.5 kB/s eta 0:00:000:010:01
Using cached typer-0.12.3-py3-none-any.whl (47 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
import lightgbm as lgb
import optuna
import mlflow
import mlflow.lightgbm
import requests
import json


# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

In [38]:
### 2

# URL de la API
api_url = 'http://127.0.0.1:8000/potability/'

# Datos para la predicción
data = {
    "pH": 10.316400384553162,
    "Hardness": 217.2668424334475,
    "Solids": 10676.508475429378,
    "Chloramines": 3.445514571005745,
    "Sulfate": 397.7549459751925,
    "Conductivity": 492.20647361771086,
    "Organic_carbon": 12.812732207582542,
    "Trihalomethanes": 72.28192021570328,
    "Turbidity": 3.4073494284238364
}

# Realizar la solicitud POST
response = requests.post(api_url, json=data)

# Mostrar la respuesta
if response.status_code == 200:
    result = response.json()
    print(f"Predicción de potabilidad: {result['potability']}")
else:
    print(f"Error al realizar la solicitud: {response.status_code}")


Predicción de potabilidad: 0


# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

Al verificar el uso de recursos desde Monitor de Actividad en relación al uso de CPU y memoria notamos que el contenedor es "liviano" en términos de recursos ya que su ejecución no marca peaks considerables en los recursos. Además, el tamaño del contenedor en el disco no es grande dado que el programa de la aplicación es básico.

Otro punto a destacar es la opción de asignar y distribuir recursos directamente desde las preferencias de docker.

Considerando que las dependencias y capas de la imagen de docker son básicas y mínimas, el contenedor es bastante liviano y rápido de ejecutar.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- **¿Cómo se diferencia Docker de una máquina virtual (VM)?**

Si bien ambas son tecnologías de virtualización, Docker es una plataforma de contenedores que permite "empaquetar" las aplicaciones y sus dependencias (librerias o modulos de python) en contenedores ligeros y portátiles, ejecutándose sobre el mismo sistema operativo anfitrión con una sobrecarga mínima. Por otro lado, una maquina virtual incluye un sistema operativo completo (como Linux, Windows, MacOS, etc) junto con la aplicación y sus dependencias, ejecutándose sobre un hipervisor (monitor de máquina virtual) que virtualiza el hardware del anfitrión. Lo anterior hace  Esto hace que las VMs sean más pesadas en términos de recursos y más lentas para iniciar, pero proporcionan un aislamiento más fuerte, ya que cada VM opera de manera independiente con su propio sistema operativo completo.

- **¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?**

La principal diferencia entre usar Docker y ejecutar una aplicación directamente en local es principalmente la portabilidad y el aislamiento. Docker permite empaquetar una aplicación junto con todas sus dependencias en un contenedor que permite que funcione consistentemente en cualquier entorno, independientemente de las configuraciones del sistema anfitrión. Lo anterior, facilita la migración y el despliegue en diferentes servicios en la nube. En contraste, ejecutar la aplicación directamente en el sistema local puede llevar a problemas de compatibilidad y configuraciones específicas del entorno, lo que puede complicar la replicación y el mantenimiento en diferentes entornos de desarrollo y producción.

- **¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?**

Docker asegura la consistencia entre diferentes entornos de desarrollo y producción al encapsular la aplicación y todas sus dependencias en contenedores. Estos contenedores incluyen todo lo necesario para ejecutar la aplicación, desde bibliotecas y configuraciones hasta el sistema operativo subyacente. Al usar imágenes de Docker para crear estos contenedores, se garantiza que la aplicación se ejecutará de la misma manera en cualquier entorno que soporte Docker, eliminando problemas de "funciona en mi máquina". Esto resulta en un comportamiento predecible y replicable, independientemente del entorno donde se despliegue la aplicación.

- **¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?**

En Docker los volúmenes se usan para gestionar la persistencia de datos (almacenamiento de forma duradera y accesible a lo largo del tiempo) permitiendo que los datos sobrevivan a los ciclos de vida de los contenedores. Los volúmenes son "directorios" en el sistema de archivos del anfitrión donde se montan en los contenedores, proporcionando una manera eficiente de almacenar datos fuera del contenedor. Con esto se garantiza que los datos no se pierdan cuando un contenedor se elimina o se recrea. Los volúmenes pueden ser gestionados por Docker, lo que facilita la administración de los datos, respaldos y la compartición de datos entre contenedores.


- **¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?**

El Dockerfile es un archivo de texto que contiene un conjunto de instrucciones para construir una imagen de Docker, especificando cómo debe configurarse el entorno de un contenedor, incluyendo el sistema operativo, las dependencias, el código de la aplicación y otros componentes necesarios. Por otro lado, el docker-compose.yml es un archivo de configuración en formato YAML utilizado por Docker Compose para definir y ejecutar aplicaciones multicontenedor. Este archivo describe los servicios, redes y volúmenes necesarios para la aplicación, permitiendo orquestar fácilmente múltiples contenedores y gestionar sus interacciones. Juntos, el Dockerfile y el docker-compose.yml facilitan la creación, despliegue y escalado de aplicaciones contenedorizadas de manera consistente y eficiente.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>